## Check GPU version

In [ ]:
# check whether GPU is provided
!nvcc --version

!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Sun Dec  6 10:09:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+

## Connect with your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 416Back   512Back  'Colab Notebooks'   DAI416Back   YS


## Setup custom darknet

In [ ]:
import os
from IPython.display import clear_output
assert os.getcwd()=='/content', 'Directory should be "/content" instead of "{}"'.format(os.getcwd())

# remove the existing folder if have
!rm -r DhakaAI_Darknet_YoloV4

# download and compile DhakaAI_Darknet_YoloV4
!git clone https://github.com/sujaysaha2906/DhakaAI_Darknet_YoloV4.git
%cd DhakaAI_Darknet_YoloV4
!make
!chmod +x ./darknet

clear_output()

## Download yolov4 pre-trained weights

In [ ]:
assert os.getcwd()=='/content/DhakaAI_Darknet_YoloV4', 'Directory should be "/content/DhakaAI_Darknet_YoloV4" instead of "{}"'.format(os.getcwd())
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

## Download custom dataset (yolo format)

In [ ]:
# copy over both datasets into the root directory of the Colab VM (comment out test.zip if you are not using a validation dataset)
!cp /mydrive/'Colab Notebooks'/DhakaAI/obj.zip ../
!cp /mydrive/'Colab Notebooks'/DhakaAI/test_R1.zip ../

# unzip the datasets and their contents so that they are now in /darknet/data/ folder
!unzip ../obj.zip -d data/
!unzip ../test_R1.zip -d data/

clear_output()

## Modify yolov4 architecture

In [ ]:
# Remove config and replace with new config

!rm /content/DhakaAI_Darknet_YoloV4/yolov4_config.py
!cp /mydrive/'Colab Notebooks'/DhakaAI/416/yolov4_config.py ./

assert os.getcwd()=='/content/DhakaAI_Darknet_YoloV4', 'Directory should be "/content/DhakaAI_Darknet_YoloV4" instead of "{}"'.format(os.getcwd())

# Run python script to create our customize yolov4_custom_train.cfg 
# and yolov4_custom_tes.cfg in folder /cfg

!python yolov4_setup.py

[INFO] Generating yolov4_custom_train.cfg successfully...
[INFO] Generating yolov4_custom_test.cfg successfully...


## Upload data file

In [ ]:
# upload the obj.data file to cloud VM from Google Drive
!cp /mydrive/'Colab Notebooks'/DhakaAI/416/obj.data  ./data

## Generate the txt file for training and testing

In [ ]:
!python generate_train.py
!python generate_test.py

# verify that the newly generated train.txt and test.txt can be seen in our darknet/data folder
!ls data/

labels	obj  obj.data  obj.names  test	test.txt  train.txt


## Create symbolic link in our Drive

Make sure that you laready created directory _YOLOv4_weight/backup_ in your Drive

In [ ]:
assert os.getcwd()=='/content/DhakaAI_Darknet_YoloV4', 'Directory should be "/content/DhakaAI_Darknet_YoloV4" instead of "{}"'.format(os.getcwd())

# delete backup folder from our 
!rm /content/DhakaAI_Darknet_YoloV4/backup -r

# create Symlinks so we can save trained weight in our Google Drive
# create folder YOLOv4_weight/back in your Drive to store trained weights
!ln -s /content/drive/'My Drive'/YOLOv4_weight/DAI416Back /content/DhakaAI_Darknet_YoloV4

## Training

In [ ]:
assert os.getcwd()=='/content/DhakaAI_Darknet_YoloV4', 'Directory should be "/content/DhakaAI_Darknet_YoloV4" instead of "{}"'.format(os.getcwd())

!./darknet detector train data/obj.data cfg/yolov4_custom_train.cfg yolov4.conv.137 -dont_show -map
#If you get CUDA out of memory adjust subdivisions above!
#adjust max batches down for shorter training above

In [ ]:
# kick off training from where it last saved
!./darknet detector train data/obj.data cfg/yolov4_custom_train.cfg /mydrive/DAI416Back/yolov4_custom_train_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 1850654, rewritten_bbox = 0.608163 % 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 139 Avg (IOU: 0.792360, GIOU: 0.786868), Class: 0.918916, Obj: 0.831196, No Obj: 0.002202, .5R: 0.894737, .75R: 0.842105, count: 38, class_loss = 1.898237, iou_loss = 82.795319, total_loss = 84.693558 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 150 Avg (IOU: 0.793404, GIOU: 0.790512), Class: 0.996859, Obj: 0.924433, No Obj: 0.005880, .5R: 0.956522, .75R: 0.826087, count: 23, class_loss = 0.574606, iou_loss = 10.037504, total_loss = 10.612110 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 161 Avg (IOU: 0.824551, GIOU: 0.815893), Class: 0.999712, Obj: 0.941846, No Obj: 0.004558, .5R: 1.000000, .75R: 1.000000, count: 3, class_loss = 0.229543, iou_loss = 0.119706, total_loss = 0.349249 
 total_bbox = 1850718, rewritten_bbox = 0.608142 % 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 139 Avg (IOU: 0.76




## Code for generating click

>```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,120000)
```

## Visualize training result

In [ ]:
assert os.getcwd()=='/content/DhakaAI_Darknet_YoloV4', 'Directory should be "/content/DhakaAI_Darknet_YoloV4" instead of "{}"'.format(os.getcwd())

# Plotting training result after 2000 epochs
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,10))
train_result = plt.imread("chart.png")

plt.axis(False)
plt.imshow(train_result)